In [1]:
import pandas as pd 
import os
import MDAnalysis as mda
import numpy as np

import nglview as nv

In [2]:
os.chdir('/home/adri/Projects/phd/bias_2')
# Local imports
from src.analysis.generate_toy_network import *

In [3]:
# Load key interactions from cluters
ms_connections = pd.read_csv('results/clustering/ms_connections.csv', index_col=0)
ms_connections.head()

,feature,coef,abs_coef,bw,group,micro_switch
535,109-93,-0.887910,0.887910,3.28-2.63,1,NaN
379,180-25,-0.693003,0.693003,180-25,1,NaN
944,25-94,0.689761,0.689761,25-2.64,1,NaN
933,26-95,0.621155,0.621155,26-2.65,1,NaN
1154,127-209,0.550139,0.550139,3.46-5.58,1,NaN


In [4]:
ACN = 'ACN_0.1_1'

In [5]:
transition_matrix_path = f'results/allosteric_network_distance/ACN_filering_thresholds/{ACN}/transition_matrices/WT_transition_matrix.csv'
allosteric_path = parse_pathway_data(transition_matrix_path)

In [6]:
allosteric_path.head()

,r1,r2,value
22,89,91,0.0105
33,93,91,0.0525
82,95,91,0.0975
146,307,303,0.0055
171,305,303,0.1580


In [7]:
micro_switch_dict = {'dry': range(130, 133),
                'npxxy': range(295, 300),
                'cwp': range(257, 261),
                'pif': [121, 201, 254],
                'sod_bs': [80, 120, 291, 292],
                'toogle_s': [258],
                '6.30': [240]}

In [8]:
ms_color_list = [19, 30, 22, 3, 11, 4, 6]

In [9]:
# load wt network
wt_network = pk.load(open('results/allosteric_network_distance/ACN_filering_thresholds/ACN_0.1_1/nw_pickles/WT/1.pickle', 'rb'))

In [10]:
# get edges and weights from network
edges = wt_network.edges()
weights = [wt_network[u][v]['weight'] for u,v in edges]

# get a list of edges with a weight larger than 0.7
background_edges = [e for e in edges if wt_network[e[0]][e[1]]['weight'] < 0.7]

# get the resid of the residues
background_edges = [[node.split(':')[1] for node in edge] for edge in background_edges]

# remove ligand (resid 1) edges
background_edges = [e for e in background_edges if e[0] != '1' and e[1] != '1']

In [11]:
signaling_df = load_signaling_df()
prefcoup_gi_mutants = signaling_df.index[signaling_df.profile == 1].astype(int).values
print(prefcoup_gi_mutants)

[ 11  17  48  50  54  55  61  65  68  71  77  82  84  86  93  98 101 109
 113 115 117 123 125 126 131 149 154 157 158 164 171 173 176 180 184 186
 190 191 197 199 202 205 213 216 217 241 246 249 250 257 268 274 279 281
 285 291 292 297 302 305 307 310 317 337]


In [12]:
acn_resids = np.unique(allosteric_path[['r1', 'r2']].values.flatten())
# scale value between 0.5 and 2
min_val = allosteric_path['value'].min()
max_val = allosteric_path['value'].max()
allosteric_path['radius'] = allosteric_path['value'].apply(lambda x: 0.1 + (x - min_val) / (max_val - min_val) * 1.5)
threshold = 0.146

In [13]:
structure_path = '/home/adri/Documents/phd/cb2_inactive.pdb'

In [19]:
# for each cluster:
wt = mda.Universe('data/raw/cb2_inactive.pdb')

# Plot individual Clusters

In [48]:

# color_dict = {1: 'cyan', 2: 'orange', 3: 'green'}
color_dict = {'positive': 7, 'negative': 1}
script = []

# Delete all molecules
script.append('mol delete all')

for group in ms_connections.group.unique():

    # load receptor
    script.append(f'mol new {{{structure_path}}} type {{pdb}} first 0 last -1 step 1 waitfor 1')

    script.append(f'mol rename top {{cluster {group}}}')

    # Modify representation of the receptor
    rep_list = ['mol selection resid < 1000',
        "mol representation NewCartoon 0.300000 10.000000 4.100000 0",
        "mol color ColorID 8",
        "mol material AOChalky",
        'mol modrep 0 top']

    for rep in rep_list:
        script.append(rep)

    # get group df
    group_df = ms_connections[ms_connections.group == group]

    # print(group_df)

    # render cylinders
    for i, row in group_df.iterrows():
        feature = row['feature']
        abs_coef = row['abs_coef']
        coef = row['coef']
        group = row['group']
        abs_coef = 1
        r1, r2 = feature.split('-')
        
        coord_list = []
        for r in [r1, r2]:
            x, y, z = wt.select_atoms(f'resid {r} and name CA').positions[0]
            coord_string = f"{{{x} {y} {z}}}"
            coord_list.append(coord_string)
        sign = 'positive' if coef > 0 else 'negative'
        color = color_dict[sign]
            
        script.append(f"draw color {color}")
        script.append(f"draw cylinder {coord_list[0]} {coord_list[1]} radius {abs_coef}")

# Render microswitches
# Load receptor
script.append(f'mol new {{{structure_path}}} type {{pdb}} first 0 last -1 step 1 waitfor 1')
script.append(f'mol rename top {{microswitches}}')
script.append('mol delrep 0 top')
color_list = ['green', 'red', 'blue', 'orange',
            'purple', 'yellow', 'grey']
vmd_color_index_list = [7, 1, 0, 3, 11, 4, 6]
# Add ms representation
for i, (ms, resids) in enumerate(micro_switch_dict.items()):

    color_idx = vmd_color_index_list[i]
    resids_str = ' '.join([str(r) for r in resids])
    script.append('mol representation VDW 1.000000 12.000000')
    script.append(f"mol color ColorID {color_idx}")
    script.append(f"mol selection name CA and resid {resids_str}")
    script.append("mol addrep top")

# Wrtite script
with open('results/clustering/ms_connections.tcl', 'w') as f:
    f.write('\n'.join(script))

In [50]:
# source /home/adri/Projects/phd/bias_2/results/clustering/ms_connections.tcl

# Plot Clusters and ACN

In [116]:
np.unique(allosteric_path[['r1', 'r2']].values.flatten())

array([  1,  24,  26,  29,  30,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  50,  51,  52,  68,
        69,  70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,
        82,  83,  84,  85,  87,  89,  91,  93,  95, 123, 125, 127, 129,
       130, 131, 133, 135, 141, 238, 240, 242, 244, 246, 248, 250, 252,
       274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286,
       287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299,
       300, 301, 302, 303, 305, 307, 309, 311, 313])

In [181]:
# for each cluster:
wt = mda.Universe('data/raw/cb2_inactive.pdb')
# color_dict = {1: 3, 2: 31, 3: 32}
color_dict = {1: 'orange', 2: 'orange2', 3: 'orange3'}

script = []

# Delete all molecules
script.append('mol delete all')

# load receptor
script.append(f'mol new {{{structure_path}}} type {{pdb}} first 0 last -1 step 1 waitfor 1')

script.append(f'mol rename top {{cluster {group}}}')

# Modify representation of the receptor
rep_list = ['mol selection resid < 1000',
    "mol representation NewCartoon 0.300000 10.000000 4.100000 0",
    "mol color ColorID 8",
    "mol material Transparent",
    'mol modrep 0 top']

for rep in rep_list:
    script.append(rep)

# Add ligand representation
rep_list = ["mol color ColorID 6",
"mol representation Surf 1.400000 0.000000",
"mol selection resname 9JU",
"mol material Glass3",
"mol addrep top",
"mol color Name",
"mol representation Licorice 0.200000 12.000000 12.000000",
"mol material AOChalky",
"mol addrep top"]

for rep in rep_list:
    script.append(rep)


script.append('draw material AOChalky')

for group in ms_connections.group.unique():
    print(group)
    count = 0
    # get group df
    group_df = ms_connections[ms_connections.group == group]

    # print(group_df)

    # render cylinders
    for i, row in group_df.iterrows():
        feature = row['feature']
        abs_coef = row['abs_coef']
        coef = row['coef']
        group = row['group']
        abs_coef = 0.5
        r1, r2 = feature.split('-')

        
        
        # if int(r1) not in acn_resids and int(r2) not in acn_resids:
        #     continue
        count += 1
        
        coord_list = []
        for r in [r1, r2]:
            x, y, z = wt.select_atoms(f'resid {r} and name CA').positions[0]
            coord_string = f"{{{x} {y} {z}}}"
            coord_list.append(coord_string)
            
        script.append(f"draw color {color_dict[group]}")
        script.append(f"draw cylinder {coord_list[0]} {coord_list[1]} radius {abs_coef}")

    print(count)
# Render ACN

for i, row in allosteric_path.iterrows():

    abs_coef = 1
    
    r1 = row['r1']
    r2 = row['r2']
    radius = row['radius']
    bc = row['value']
    
    coord_list = []
    failed = False
    for r in [r1, r2]:
        try:
            selection = wt.select_atoms(f'resid {int(r)} and name CA')
            if selection.n_atoms != 1:
                failed = True
                continue
            x, y, z = selection.positions[0]
            
        except mda.SelectionError as e:
            failed = True
            print(e)
            continue
        
        coord_string = f"{{{x} {y} {z}}}"
        coord_list.append(coord_string)
        
    if failed:
        continue

    if bc > threshold:
        script.append(f"draw color silver")
    else:
        script.append(f"draw color white")
    
    
    script.append(f"draw cylinder {coord_list[0]} {coord_list[1]} radius {radius}")

# Plot background network
# We need to load the receptor again because there can only be one drawing material per molecule
script.append(f'mol new {{{structure_path}}} type {{pdb}} first 0 last -1 step 1 waitfor 1')
script.append(f'mol delrep 0 top')

script.append(f"draw color white")
script.append(f"draw material Transparent")
for r1, r2 in background_edges:
    coord_list = []
    for r in [r1, r2]:
        x, y, z = wt.select_atoms(f'resid {r} and name CA').positions[0]
        coord_string = f"{{{x} {y} {z}}}"
        coord_list.append(coord_string)
    script.append(f"draw cylinder {coord_list[0]} {coord_list[1]} radius 0.05")

# Wrtite script
output_path = f'results/clustering/{ACN}_clusters.tcl'
with open(output_path, 'w') as f:
    f.write('\n'.join(script))

1
20
2
20
3
20


In [174]:
print(f"source {os.path.abspath(output_path)}")

source /home/adri/Projects/phd/bias_2/results/clustering/ACN_0.1_1_clusters.tcl


In [35]:
# Render using nglview
# load receptor
view = nv.show_structure_file(structure_path)
view.camera = 'orthographic'

# Modify representation of the receptor
view.clear_representations()
view.add_cartoon(selection='1-999', color='white')
view.add_surface(selection='[9JU]', color='grey')

# render ms connections
for group in ms_connections.group.unique():
    count = 0
    # get group df
    group_df = ms_connections[ms_connections.group == group]

    # print(group_df)

    # render cylinders
    for i, row in group_df.iterrows():
        feature = row['feature']
        abs_coef = row['abs_coef']
        coef = row['coef']
        group = row['group']
        abs_coef = 0.5
        r1, r2 = feature.split('-')

        
        
        # if int(r1) not in acn_resids and int(r2) not in acn_resids:
        #     continue
        count += 1
        
        coord_list = []
        for r in [r1, r2]:
            x, y, z = wt.select_atoms(f'resid {r} and name CA').positions[0]
            coord_list.append([x, y, z])
            
        if group == 1:
            # dark orange in rgb format
            color = (1, 0.25, 0)
        elif group == 2:
            # light orage in rgb format
            color =  (1, 0.75, 0)
        elif group == 3:
            # medium orange in rgb format
            color = (1, 0.5, 0)
        else:
            color = (1, 1, 1)
            
        view.shape.add_cylinder(coord_list[0], coord_list[1], color, abs_coef)

# add ACN
shape = nv.shape.Shape(view=view)
for i, row in allosteric_path.iterrows():

    abs_coef = 1
    
    r1 = row['r1']
    r2 = row['r2']
    radius = row['radius']
    bc = row['value']
    
    coord_list = []
    failed = False
    for r in [r1, r2]:
        try:
            selection = wt.select_atoms(f'resid {int(r)} and name CA')
            if selection.n_atoms != 1:
                failed = True
                continue
            x, y, z = selection.positions[0]
            
        except mda.SelectionError as e:
            failed = True
            print(e)
            continue
        
        coord_list.append([x, y, z])
        
    if failed:
        continue

    if bc > threshold:
        # Dark blue in rgb format
        color= (0, 1, 1)
    else:
        # cyan in rgb format
        color = (0, 0, 0.5)

    shape.add_cylinder(coord_list[0], coord_list[1], color, radius)

# add background network
for r1, r2 in background_edges:
    coord_list = []
    for r in [r1, r2]:
        x, y, z = wt.select_atoms(f'resid {r} and name CA').positions[0]
        coord_list.append([x, y, z])
    shape.add_cylinder(coord_list[0], coord_list[1], (1, 1, 1), 0.05)

In [ ]:
embed_state = view._get_embed_state()
# save embded state as a pickle file
pk.dump(embed_state, open(f'results/clustering/{ACN}_clusters.pickle', 'wb'))

# Plot ACN and microswitches

In [14]:
# for each cluster:
wt = mda.Universe('data/raw/cb2_inactive.pdb')
# color_dict = {1: 3, 2: 31, 3: 32}
color_dict = {1: 'orange', 2: 'orange2', 3: 'orange3'}

script = []

# Delete all molecules
script.append('mol delete all')

# load receptor
script.append(f'mol new {{{structure_path}}} type {{pdb}} first 0 last -1 step 1 waitfor 1')

# Modify representation of the receptor
rep_list = ['mol selection resid < 1000',
    "mol representation NewCartoon 0.300000 10.000000 4.100000 0",
    "mol color ColorID 8",
    "mol material Transparent",
    'mol modrep 0 top']

for rep in rep_list:
    script.append(rep)


# Add ligand representation
rep_list = ["mol color ColorID 6",
"mol representation Surf 1.400000 0.000000",
"mol selection resname 9JU",
"mol material Glass3",
"mol addrep top",
"mol color Name",
"mol representation Licorice 0.200000 12.000000 12.000000",
"mol material AOChalky",
"mol addrep top"]

for rep in rep_list:
    script.append(rep)

# add prefcoup mutants
rep_list = ["mol color ColorID 32",
"mol representation VDW 0.750000 12.000000",
f"mol selection name CA and resid {' '.join([str(m) for m in prefcoup_gi_mutants])}",
"mol material AOChalky",
"mol addrep top"]

for rep in rep_list:
    script.append(rep)

script.append('mol material AOChalky')

for (ms, resids), color in zip(micro_switch_dict.items(), ms_color_list):

    resids_str = ' '.join([str(r) for r in resids])
    script.append('mol representation VDW 1.000000 12.000000')
    script.append(f"mol color colorID {color}")
    script.append(f"mol selection name CA and resid {resids_str}")
    script.append("mol addrep top")
    
# Render ACN
script.append(f"draw material AOChalky")
for i, row in allosteric_path.iterrows():

    abs_coef = 1
    
    r1 = row['r1']
    r2 = row['r2']
    radius = row['radius']
    bc = row['value']
    
    coord_list = []
    failed = False
    for r in [r1, r2]:
        try:
            selection = wt.select_atoms(f'resid {int(r)} and name CA')
            if selection.n_atoms != 1:
                failed = True
                continue
            x, y, z = selection.positions[0]
            
        except mda.SelectionError as e:
            failed = True
            print(e)
            continue
        
        coord_string = f"{{{x} {y} {z}}}"
        coord_list.append(coord_string)
        
    if failed:
        continue

    if bc > threshold:
        script.append(f"draw color silver")
    else:
        script.append(f"draw color white")
    
    
    script.append(f"draw cylinder {coord_list[0]} {coord_list[1]} radius {radius}")

# Plot background network
# We need to load the receptor again because there can only be one drawing material per molecule
script.append(f'mol new {{{structure_path}}} type {{pdb}} first 0 last -1 step 1 waitfor 1')
script.append(f'mol delrep 0 top')

script.append(f"draw color white")
script.append(f"draw material Transparent")
for r1, r2 in background_edges:
    coord_list = []
    for r in [r1, r2]:
        x, y, z = wt.select_atoms(f'resid {r} and name CA').positions[0]
        coord_string = f"{{{x} {y} {z}}}"
        coord_list.append(coord_string)
    script.append(f"draw cylinder {coord_list[0]} {coord_list[1]} radius 0.05")

script.append('display resetview')

# Wrtite script
output_path = f'results/clustering/{ACN}_ms.tcl'
with open(output_path, 'w') as f:
    f.write('\n'.join(script))

In [15]:
print(f"source {os.path.abspath(output_path)}")

source /home/adri/Projects/phd/bias_2/results/clustering/ACN_0.1_1_ms.tcl


In [14]:
# replicate this vmd visualization using nglview


In [15]:
structure_path

'/home/adri/Documents/phd/cb2_inactive.pdb'

In [16]:
prefcoup_str = ' '.join([str(resid) for resid in prefcoup_gi_mutants])

In [17]:
ms_str_color_list = ['green', 'red', 'cyan', 'orange',
            'purple', 'yellow', 'grey']

In [20]:
# load receptor
view = nv.show_structure_file(structure_path)
view.camera = 'orthographic'

# Modify representation of the receptor
view.clear_representations()
view.add_cartoon(selection='1-999', color='white')

view.add_surface(selection='[9JU]', color='grey')


# add prefcoup mutants
view.add_spacefill(selection=f'(.CA) and ({prefcoup_str})', color='orange')

# add microswitches

for (ms, resids), color in zip(micro_switch_dict.items(), ms_str_color_list):
    resids_str = ', '.join([str(r) for r in resids])
    view.add_spacefill(selection=f'(.CA) and ({resids_str})', color=color)

# add ACN
shape = nv.shape.Shape(view=view)
for i, row in allosteric_path.iterrows():

    abs_coef = 1
    
    r1 = row['r1']
    r2 = row['r2']
    radius = row['radius']
    bc = row['value']
    
    coord_list = []
    failed = False
    for r in [r1, r2]:
        try:
            selection = wt.select_atoms(f'resid {int(r)} and name CA')
            if selection.n_atoms != 1:
                failed = True
                continue
            x, y, z = selection.positions[0]
            
        except mda.SelectionError as e:
            failed = True
            print(e)
            continue
        
        coord_list.append([x, y, z])
        
    if failed:
        continue

    if bc > threshold:
        color=(0.5, 0.5, 0.5)
    else:
        color = (1, 1, 1)


    shape.add_cylinder(coord_list[0], coord_list[1], color, radius)

# add background network
for r1, r2 in background_edges:
    coord_list = []
    for r in [r1, r2]:
        x, y, z = wt.select_atoms(f'resid {r} and name CA').positions[0]
        coord_list.append([x, y, z])
    shape.add_cylinder(coord_list[0], coord_list[1], (1, 1, 1), 0.05)

In [23]:
embed_state = view._get_embed_state()
# save embded state as a pickle file
pk.dump(embed_state, open(f'results/clustering/{ACN}_ms.pickle', 'wb'))